In [120]:
import pandas as pd
from pandas import DataFrame as df
import os

In [121]:
patinfo = pd.read_csv('./my_data/sepsis_merged_data.csv')
concat_data = pd.read_csv('./my_data_concat/all_parameters.csv')

In [122]:
concat_data = concat_data.drop(['Unnamed: 0'], axis=1)
concat_data = concat_data.dropna(subset=['stay_id'],axis=0)

In [123]:
concat_data = concat_data.drop(['stay_id'], axis=1)

In [124]:
sepsis_pat = patinfo[patinfo['y']==1]

In [125]:
concat_data['year'] = concat_data['t_time'].map(lambda x: x[:4])

In [126]:
concat_data

,subject_id,hadm_id,t_time,respir_rate,heart_rate,blood_oxy_satur,sys_blood_pressure,temperature,pH_value,wbc_count,CO2_part_press,dias_blood_pressure,year
0,10000032,29079034,2180-07-23 14:00:00,22.5,92.0,97.5,89.5,98.7,NaN,NaN,NaN,53.5,2180
1,10000032,29079034,2180-07-23 15:00:00,23.0,94.0,97.0,88.0,NaN,NaN,NaN,NaN,56.0,2180
2,10000032,29079034,2180-07-23 16:00:00,21.0,105.0,94.0,91.0,NaN,NaN,NaN,NaN,55.0,2180
3,10000032,29079034,2180-07-23 17:00:00,20.0,97.0,95.0,95.0,98.7,NaN,NaN,NaN,58.0,2180
4,10000032,29079034,2180-07-23 18:00:00,21.0,100.0,95.0,86.0,NaN,NaN,NaN,NaN,53.0,2180
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841830,15612570,26152435,2114-04-05 21:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,2114
7841831,16721241,22649116,2183-08-30 05:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,125.0,2183
7841832,18986646,25231593,2146-12-09 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,82.0,2146
7841833,19635953,29856471,2180-11-23 11:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.0,2180


In [127]:
sepsis_pat['year'] = sepsis_pat['chartdate'].map(lambda x: x[:4])
sepsis_pat

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,subject_id,hadm_id,seq_num,y,chartdate,age,year
1152,10216556.0,23888667.0,5.0,1,2131-06-14,61,2131
2218,10149400.0,20346295.0,2.0,1,2119-01-04,72,2119
2396,10332371.0,29819591.0,2.0,1,2149-02-28,69,2149
2686,10246670.0,23695400.0,2.0,1,2114-12-10,44,2114
2743,10004401.0,25777141.0,2.0,1,2144-06-05,85,2144
...,...,...,...,...,...,...,...
367708,12938211.0,21413105.0,1.0,1,2202-08-21,55,2202
367709,11662929.0,22058796.0,6.0,1,2166-08-05,52,2166
367710,18363890.0,24322267.0,2.0,1,2161-08-22,71,2161
367711,16545869.0,29143784.0,1.0,1,2169-09-16,76,2169


In [128]:
result = pd.merge(sepsis_pat, concat_data, on=['subject_id', 'hadm_id', 'year'], how='left')

In [129]:
result = result.dropna(subset=['t_time'])

In [130]:
sum(list(result.isna().sum()))/9

394586.44444444444

In [131]:
sub_id = list(set(result.subject_id.values))

In [132]:
len_sub = [len(result[result['subject_id']==i]) for i in sub_id]

In [133]:
mean_nan = [sum(list(result[result['subject_id']==i].isna().sum()))/9 for i in sub_id]

In [134]:
cal_data = df({'subject_id':sub_id, 'len_sub':len_sub, 'nan_mean':mean_nan})

In [135]:
cal_data['nan'] = cal_data['nan_mean']/cal_data['len_sub']

In [136]:
cal_data.sort_values(by=['len_sub'], ascending=False)

,subject_id,len_sub,nan_mean,nan
2879,14923562.0,4044,1500.777778,0.371112
3554,10304606.0,3260,1303.333333,0.399796
1356,19131048.0,3153,1269.222222,0.402544
753,17876390.0,2927,1051.222222,0.359147
1679,14765354.0,2856,1109.000000,0.388305
...,...,...,...,...
345,15991401.0,2,0.888889,0.444444
2920,16529283.0,2,0.666667,0.333333
3459,11287462.0,2,0.444444,0.222222
3385,17636096.0,2,0.888889,0.444444


In [137]:
cal_data.sort_values(by=['nan'])

,subject_id,len_sub,nan_mean,nan
2338,18330363.0,3,0.444444,0.148148
3224,14211509.0,4,0.777778,0.194444
1475,12569468.0,21,4.111111,0.195767
2375,14537549.0,16,3.222222,0.201389
3459,11287462.0,2,0.444444,0.222222
...,...,...,...,...
1155,19032335.0,49,32.666667,0.666667
2258,16634427.0,3,2.000000,0.666667
3078,17938576.0,3,2.111111,0.703704
48,11747400.0,22,15.777778,0.717172


In [138]:
result.isna().sum()

subject_id                  0
hadm_id                     0
seq_num                     0
y                           0
chartdate                   0
age                         0
year                        0
t_time                      0
respir_rate              7663
heart_rate               5402
blood_oxy_satur         15353
sys_blood_pressure      23963
temperature            687204
pH_value               916493
wbc_count              931925
CO2_part_press         939272
dias_blood_pressure     24003
dtype: int64

In [139]:
cal_data[cal_data['len_sub']>100].sort_values(by=['nan'])

,subject_id,len_sub,nan_mean,nan
3110,16644308.0,192,50.444444,0.262731
3019,12515381.0,242,65.111111,0.269054
3601,19340496.0,525,143.888889,0.274074
1687,14691641.0,294,81.111111,0.275888
1977,16797598.0,465,130.666667,0.281004
...,...,...,...,...
10,12648465.0,484,251.555556,0.519743
889,15623906.0,378,197.000000,0.521164
1236,14879136.0,239,128.888889,0.539284
2937,17905563.0,548,299.222222,0.546026


In [140]:
result.groupby(by=['subject_id', 'hadm_id']).count().sort_values(by=['seq_num'],ascending=False)

,,seq_num,y,chartdate,age,year,t_time,respir_rate,heart_rate,blood_oxy_satur,sys_blood_pressure,temperature,pH_value,wbc_count,CO2_part_press,dias_blood_pressure
subject_id,hadm_id,,,,,,,,,,,,,,,
14923562.0,23365149.0,4044,4044,4044,4044,4044,4044,4028,4025,4019,4015,1261,676,315,534,4016
10304606.0,25532105.0,3260,3260,3260,3260,3260,3260,3258,3244,3254,3252,892,158,196,104,3252
14765354.0,27368831.0,2856,2856,2856,2856,2856,2856,2844,2853,2835,2820,801,300,204,246,2820
15032392.0,22592769.0,2766,2766,2766,2766,2766,2766,2744,2748,2746,2744,906,284,244,282,2744
18777918.0,24753501.0,2739,2739,2739,2739,2739,2739,2739,2736,2730,2739,1533,585,360,549,2739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18860049.0,28690706.0,2,2,2,2,2,2,2,2,0,2,0,1,1,0,2
16529283.0,27536642.0,2,2,2,2,2,2,2,2,2,2,2,0,0,0,2
15991401.0,28873698.0,2,2,2,2,2,2,2,2,2,2,0,0,0,0,2


In [142]:
best = result[result['subject_id']==14923562.0]
best = best[best['hadm_id']==23365149.0]

In [146]:
best = best.sort_values(by=['t_time'])

In [148]:
best[708:].to_csv('./my_data_concat/best.csv', index=False)

In [149]:
best = best[708:]

In [151]:
na_list = [i/len(best) for i in list(best.isna().sum())]

In [ ]:
for i in range(len(na_list)):
    print("{0} : {1}".format(best.columns[i], round(na_list[i],3)))

In [156]:
len(best)

3336